In [1]:
import datetime
import logging
import time
from selenium import webdriver
import json
import re
from pathlib import Path
import pandas as pd

In [4]:
def Convert2Tera(raw):
    for miner in raw:
        if miner[2] == 'GB':
            miner[1] = str(round(float(miner[1]) / 1000, 2))
            miner[2] = 'TB'
    return raw

def SanityCheck(raw):
    cache = []
    for i, miner in enumerate(raw):
        if miner[-3] != u'在线':
            cache.append(i)
    for offlines in cache:
        raw.pop(int(offlines))
    return raw

def Convert2Dict(raw):
    kv = {}
    for miner in raw:
        kv[miner[0]] = miner[1]
    return kv

def toLowerCase(string):
    return "".join(chr(ord(c) + 32) if 65 <= ord(c) <= 90 else c for c in string)

def FormatString(raw):
    for miner in raw:
        miner[0] = miner[0].lower()
    return raw

def ParsebyJson(raw, save_dir):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
    save_dir = Path(save_dir)
    with open('worker_keywords.json') as json_file:
        kv = json.load(json_file)
        for key in kv:
            for worker in raw:
                if re.search(key, worker):
                    kv[key].append(float(raw[worker]))
    for key in kv:
        kv[key] = sum(kv[key])
    keys = pd.DataFrame(data=raw, index=['volume'])
    keys.to_csv(str(save_dir /  Path(f'{current_time}.csv')))

In [5]:
logging.basicConfig(level=logging.NOTSET)

options = webdriver.ChromeOptions()
options.add_argument(r"user-data-dir=C:\Users\tiwang\AppData\Local\Google\Chrome\User Data")
options.add_argument(r"profile-directory=Profile 1")
driver = webdriver.Chrome(r'./chromedriver.exe', chrome_options=options)

driver.get("https://www.hpool.com/center/machine")
time.sleep(2)

result = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div[3]/div[2]/div/table/tbody').text
driver.quit()
result = result.splitlines()
result = [x.split(' ') for x in result]

C:\Users\tiwang\anaconda3\envs\chialogger\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:58474/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"extensions": [], "args": ["user-data-dir=C:\\Users\\tiwang\\AppData\\Local\\Google\\Chrome\\User Data", "profile-directory=Profile 1"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"extensions": [], "args": ["user-data-dir=C:\\Users\\tiwang\\AppData\\Local\\Google\\Chrome\\User Data", "profile-directory=Profile 1"]}}}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:58474
DEBUG:urllib3.connectionpool:http://127.0.0.1:58474 "POST /session HTTP/1.1" 200 748
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.re

In [6]:
result = Convert2Tera(result)

In [7]:
result = SanityCheck(result)

In [8]:
result = FormatString(result)

In [9]:
result = Convert2Dict(result)

In [10]:
ParsebyJson(result, './')

In [11]:
from glob import glob


In [20]:
def DailyAverage():
    """
    :param date: date formatted as yyyy-mm-dd-hh-mm
    """
    current_time = datetime.datetime.now().strftime('%Y-%m-%d')
    today = glob(f'./{current_time}-*.csv')
    cache = []
    for file in today:
        cache.append(pd.read_csv(file))
    full = pd.concat(cache, join='outer', axis=0, ignore_index=True)
    full = full.drop('Unnamed: 0', axis=1)
    full = full.fillna(0)
    print(full)
    full = full.sum(axis=0) / full.shape[0]
    full = pd.DataFrame(full, columns=['Average']).T
    full.to_csv(str(Path.cwd() /  Path(f'{current_time}.csv')))
    return full

In [21]:
DailyAverage()


   dualdragon  eric  josuke  xinghuo  pornlordsu
0        0.51  2.08    0.00     0.00        0.00
1        0.51  2.18    0.00     0.00        0.00
2        0.51  2.28    1.29     1.39        0.00
3        0.51  2.38    1.19    10.99       11.29
4        0.51  2.38    1.19    10.99       11.29


,dualdragon,eric,josuke,xinghuo,pornlordsu
Average,0.51,2.26,0.734,4.674,4.516
